In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
DEGREE_COLUMNS = [
    'university',
    'name',
    'university_url',
    'modality',
    'price',
    'location',
    'credits'
]

TYPE_NAME = 'grados_ciencia_datos'

In [3]:
LISTA_GRADOS_URL = 'https://www.gradomania.com/index.php?texto_busqueda=ciencia+de+datos&pagina=cursos'

In [4]:
PATH = 'https://www.gradomania.com/'

In [5]:
DEGREE_LIST = []
DEGREE_ERRORS = []

In [6]:
def get_text(value):
    return value[0].getText(strip=True) if len(value) else ''

In [7]:
def get_degree(url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, 'html.parser')

    try:
        name = soup.select('.valor.titulacion')[0].getText(strip=True)
        university = soup.select('.valor.imparte')[0].getText(strip=True)
        university_url = soup.select('.valor.imparte a')[0].get('href')
        modality = soup.select('.valor.modalidad')[0].getText(strip=True)
        price = soup.select('.valor.precio')[0].getText(strip=True)
        location = soup.select('.valor.lugar')[0].getText(strip=True)
        credits = soup.select('.valor.duracion')

        degree_data = {
            'name': name,
            'university': university,
            'university_url': university_url,
            'modality': modality,
            'price': price,
            'location': location,
            'credits': get_text(credits)
        }

        DEGREE_LIST.append(degree_data)
    except:
        DEGREE_ERRORS.append(url)

In [8]:
def get_degrees(url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    degrees = soup.select('.ficha_evento_texto > article > a:last-of-type')

    for degree in degrees:
        degree_url = degree.get('href')
        get_degree(degree_url)

In [9]:
def get_degree_pages(url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    lista = soup.select('.paginador_lista ul > li > a')
    
    if len(lista):
        for elem in lista:
            page_url = elem.get('href')
            get_degrees(PATH + page_url)
    else:
        get_degrees(url)

In [10]:
get_degree_pages(LISTA_GRADOS_URL)

/Users/elenatorro/.local/share/virtualenvs/unknowniversity-5YM2mFLK/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gradomania.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/elenatorro/.local/share/virtualenvs/unknowniversity-5YM2mFLK/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gradomania.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/elenatorro/.local/share/virtualenvs/unknowniversity-5YM2mFLK/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gradomania.com'. Adding certificate verif

In [11]:
degrees_df = pd.DataFrame(DEGREE_LIST, columns=DEGREE_COLUMNS)

In [12]:
degrees_df.head()

,university,name,university_url,modality,price,location,credits
0,Escola Técnica Superior d'Enginyería de Teleco...,Título Oficial de Grado en Ciencia e Ingenierí...,https://www.gradomania.com/escola-tecnica-supe...,Presencial,Información no disponible,BarcelonaEspaña,240 ECTS
1,Facultat d'Informàtica de Barcelona (FIB) - Un...,Título Oficial de Grado en Ciencia e Ingenierí...,https://www.gradomania.com/facultat-dinformati...,Presencial,Información no disponible,BarcelonaEspaña,240 ECTS
2,Facultat de Matemàtiques i Estadística (FME) -...,Título Oficial de Grado en Ciencia e Ingenierí...,https://www.gradomania.com/facultat-de-matemat...,Presencial,Información no disponible,"Campus Sur de la Diagonal C. Pau Gargallo, 14...",240 ECTS
3,Escuela Técnica Superior de Ingenieros Agrónom...,Título Oficial de Grado en Ciencia de Datos.,https://www.gradomania.com/escuela-tecnica-sup...,Presencial,Información no disponible,Navarra / NafarroaEspaña,240 ECTS
4,Estudios Oficiales UOC - Universitat Oberta de...,Grado de Ciencia de Datos Aplicada (Applied Da...,https://www.gradomania.com/estudios-oficiales-...,Online,Información no disponible,Se imparte Online,240 ECTS


In [13]:
degrees_df.to_csv('{}.csv'.format(TYPE_NAME), index=False)

In [14]:
DEGREE_ERRORS

[]